In [1]:
import sys
from PIL import Image as Img
import json
from transformers import AutoModel
import numpy as np
import torch
from pathlib import Path

In [2]:
file_path = 'LoveHina_vol01_fixed_order'

In [3]:
page_folder_path = 'LoveHina_vol01'

In [4]:
f = open(f'{file_path}.json')

annotation_data = json.load(f)

In [5]:
page_split_data = []

In [6]:
def sort_lines(lines):
    sorted_lines = sorted(lines, key=lambda x:x['read_order'])
    return sorted_lines


In [7]:
def get_index_str(index:int):
    return ('0' if index < 100 else '') + ('0' if index < 10 else '') + str(index)

In [8]:
def handle_page_splitting(index, data_path, width, height):
    index_str = get_index_str(index)

    img_path = data_path + '/' + index_str + '.jpg'

    pil_img = Img.open(img_path)

    left_page = pil_img.crop((0, 0, width/2, height))

    right_page = pil_img.crop((width/2, 0, width, height))

    save_data_path = data_path + '_page_split'

    Path(save_data_path).mkdir(parents=True, exist_ok=True)

    right_page_save_path = save_data_path+'/'+get_index_str(2*index)+'.jpg'
    left_page_save_path = save_data_path+'/'+get_index_str(2*index+1)+'.jpg'

    right_page.save(right_page_save_path)
    left_page.save(left_page_save_path)

    return get_index_str(2*index)+'.jpg', get_index_str(2*index+1)+'.jpg'



In [9]:
for i, page in enumerate(annotation_data):
    width = int(page['attrib']['width'])
    height = int(page['attrib']['height'])

    right_page_save_path, left_page_save_path = handle_page_splitting(i, page_folder_path, width, height)

    frames_left = []
    frames_right = []

    lines_left = []
    lines_right = []

    for frame in page['frames']:
        if int(frame['xmin']) < width/2:
            frames_left.append(frame)
        else:
            frame['xmin'] = str(int(int(frame['xmin']) - width/2))
            frame['xmax'] = str(int(int(frame['xmax']) - width/2))
            frames_right.append(frame)

    for line in page['lines']:
        if int(line['attrib']['xmin']) < width/2:
            lines_left.append(line)
        else:
            line['attrib']['xmin'] = str(int(int(line['attrib']['xmin']) - width/2))
            line['attrib']['xmax'] = str(int(int(line['attrib']['xmax']) - width/2))
            lines_right.append(line)

    page_dict_left = {}

    page_dict_right = {}

    page_dict_right['index'] = i * 2 

    page_dict_right['frames'] = frames_right

    page_dict_right['lines'] = sort_lines(lines_right)
    
    page_dict_right['path'] = right_page_save_path

    page_split_data.append(page_dict_right)

    page_dict_left['index'] = i * 2 + 1

    page_dict_left['frames'] = frames_left 

    page_dict_left['lines'] = sort_lines(lines_left)

    page_dict_left['path'] = left_page_save_path

    page_split_data.append(page_dict_left)

    

    

    

In [10]:
with open(f'{file_path}_page_split.json', 'w', encoding='utf-8') as f:
    json.dump(page_split_data, f, ensure_ascii=False, indent=2)